<a href="https://colab.research.google.com/github/524429264/-/blob/master/pressure_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold




# 设置随机种子以确保可重复性
np.random.seed(42)
tf.random.set_seed(42)

# 数据加载和预处理函数
def load_data(data_dir):
    images = []
    labels = []
    for sample in os.listdir(data_dir):
        sample_dir = os.path.join(data_dir, sample)
        if os.path.isdir(sample_dir):
            sample_images = []
            for i in range(1, 21):
                img_path = os.path.join(sample_dir, f'image_{i}.jpg')
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                sample_images.append(img_array)
            images.append(np.array(sample_images))

            with open(os.path.join(sample_dir, 'label.txt'), 'r') as f:
                label = float(f.read().strip())
            labels.append(label)

    return np.array(images), np.array(labels)

# 数据增强
def augment_data(images, labels):
    augmented_images = []
    augmented_labels = []

    for image_seq, label in zip(images, labels):
        augmented_images.append(image_seq)
        augmented_labels.append(label)

        # 水平翻转
        flipped_seq = np.flip(image_seq, axis=2)
        augmented_images.append(flipped_seq)
        augmented_labels.append(label)

        # 随机亮度变化
        brightness_seq = tf.image.random_brightness(image_seq, max_delta=0.2)
        augmented_images.append(brightness_seq.numpy())
        augmented_labels.append(label)

    return np.array(augmented_images), np.array(augmented_labels)

# 构建改进的模型
def build_model(input_shape):
    base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=input_shape[1:])

    model = models.Sequential([
        layers.TimeDistributed(base_model),
        layers.GlobalAveragePooling3D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    return model

# Huber损失函数
def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    abs_error = tf.abs(error)
    quadratic = tf.minimum(abs_error, delta)
    linear = abs_error - quadratic
    return 0.5 * tf.square(quadratic) + delta * linear

# 学习率调度
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    return lr

# 主函数
def main():
    # 加载数据
    train_dir = 'drive/My Drive/explosion_data'
    test_dir = 'drive/My Drive/explosion_testdata'

    X_train, y_train = load_data(train_dir)
    X_test, y_test = load_data(test_dir)

    # 数据增强
    X_train, y_train = augment_data(X_train, y_train)

    # 数据标准化
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # K折交叉验证
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    fold_no = 1
    for train_index, val_index in kfold.split(X_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        model = build_model(X_train.shape[1:])

        optimizer = optimizers.Adam(learning_rate=1e-3)
        model.compile(optimizer=optimizer, loss=huber_loss, metrics=['mae'])

        lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
        early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

        history = model.fit(
            X_train_fold, y_train_fold,
            validation_data=(X_val_fold, y_val_fold),
            epochs=50,
            batch_size=8,
            callbacks=[lr_scheduler, early_stopping]
        )

        # 评估模型
        test_loss, test_mae = model.evaluate(X_test, y_test)
        print(f'Fold {fold_no} - Test MAE: {test_mae:.4f}')

        # 预测测试集
        y_pred = model.predict(X_test)
        print(f'Fold {fold_no} - Predicted value: {y_pred[0][0]:.4f}, Actual value: {y_test[0]:.4f}')

        fold_no += 1

if __name__ == '__main__':
    main()

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50
